In [87]:
from sklearn import svm
# from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from nltk.stem import RSLPStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import classification_report, confusion_matrix

import pandas as pd
import csv
import numpy as np
# Import supplementary visualization code visuals.py
import visuals as vs

import re

In [88]:
!ls

RandomForest.pickle         main.pyc
__pycache__                 model.ipynb
analysis.Rmd                playlist_processor.py
analysis.pdf                protest.csv
analysis_files              pt_stopwords.txt
brz_dictatorship.csv        visuals.py
brz_dictatorship_target.csv whosampled_scraper.py
jovem_guarda.csv


In [89]:
df = pd.read_csv('brz_dictatorship.csv', encoding='utf-8')
df = df.dropna(axis=0)
display(df.head())

stopwords = open("pt_stopwords.txt").readlines()
stopwords = list(map(str.strip, stopwords))

,Unnamed: 0,song_sp_uri,song_name,song_isrc,song_popularity,song_lyrics,class,danceability,energy,key,...,valence,tempo,id,duration_ms,time_signature,artist_genres,artist_name,artist_photo,artist_popularity,artist_sp_followers
0,0,spotify:track:0VUgbCK0k8QWGpLiEV8YYZ,Cálice,BRPGD7800015,49,"pai , afasta mim calice pai , afasta mim calic...",Protest,0.596,0.372,4,...,0.293,123.125,0VUgbCK0k8QWGpLiEV8YYZ,241867,4,bossa nova,Chico Buarque,https://i.scdn.co/image/c1a6ae9e79561abeb0bd97...,63,539597
1,1,spotify:track:2GAFZG9Z7UGS1iMm4Idrnr,Apesar De Você,BRPGD7800024,57,amanha vai outro dia amanha vai outro dia aman...,Protest,0.568,0.574,4,...,0.680,107.820,2GAFZG9Z7UGS1iMm4Idrnr,235547,4,bossa nova,Chico Buarque,https://i.scdn.co/image/c1a6ae9e79561abeb0bd97...,63,539597
2,2,spotify:track:06ND7qqsmIRCuWdQNQIlTE,Roda-Viva,BRSGL6800006,53,dias gente sente partiu morreu gente estancou ...,Protest,0.502,0.512,10,...,0.651,133.174,06ND7qqsmIRCuWdQNQIlTE,233400,4,bossa nova,Chico Buarque,https://i.scdn.co/image/c1a6ae9e79561abeb0bd97...,63,539597
3,3,spotify:track:3Ig3k8enaBZy7cohqfnWVz,Como nossos pais,BRWMB9705419,27,nao quero falar grande amor coisas aprendi dis...,Protest,0.463,0.337,8,...,0.269,96.559,3Ig3k8enaBZy7cohqfnWVz,280627,4,bossa nova,Belchior,https://i.scdn.co/image/22365c7789eb990937bb71...,56,297473
4,4,spotify:track:3AQg8DR76bH9Ko3mZXuSWK,Mosca Na Sopa,BRMCA7300117,44,mosca pousou sopa mosca pintou pra abusar mosc...,Protest,0.609,0.760,2,...,0.880,92.022,3AQg8DR76bH9Ko3mZXuSWK,239187,4,brazilian psychedelic,Raul Seixas,https://i.scdn.co/image/ad67b61dd80b071a2ac908...,62,825295


In [91]:
stemmer = RSLPStemmer()
df['cleaned'] = df['song_lyrics'].astype('str')
df['cleaned'] = df['cleaned'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split()]).lower())

dtype('float64')

In [113]:
vectorizer = TfidfVectorizer(min_df= 3, stop_words=stopwords, sublinear_tf=True, norm='l2', ngram_range=(1, 2))
final_features = vectorizer.fit_transform(df['cleaned']).toarray()
final_features.shape

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [106]:
# this block is to split the dataset into training and testing set 
X = final_features

Y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

# instead of doing these steps one at a time, we can use a pipeline to complete then all at once
pipeline = Pipeline([
#                      ('chi',  SelectKBest(chi2, k=1200)),
                     ('clf', RandomForestClassifier())])

# fitting our model and save it in a pickle for later use
model = pipeline.fit(X_train, y_train)
# with open('RandomForest.pickle', 'wb') as f:
#     pickle.dump(model, f)

ytest = np.array(y_test)

# confusion matrix and classification report(precision, recall, F1-score)
print(classification_report(ytest, model.predict(X_test)))
print(confusion_matrix(ytest, model.predict(X_test)))


              precision    recall  f1-score   support

Jovem Guarda       0.50      0.75      0.60         4
     Protest       0.50      0.25      0.33         4

   micro avg       0.50      0.50      0.50         8
   macro avg       0.50      0.50      0.47         8
weighted avg       0.50      0.50      0.47         8

[[3 1]
 [3 1]]


/Users/gabrielsaruhashi/anaconda2/envs/py37/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
